# the `teeth.array` module: explore and tokenize text

## intro

text is deep. its size is epic. its rules are murky. it summons things from across timespace. it has to be chaotic, so everything can fit.

even just splitting text into tokens is hard. there are rules, but those almost always have exceptions, and the exceptions tend to multiply. it's helpful to experiment and adjust, in small starts.

these data structures are intended as a tool to do just that.

## *technical detail

this notebook is bundled with source; we need to make sure the cells below can import from it:

In [ ]:
import sys
import os

package_path = os.path.abspath( '..' )

if package_path not in sys.path:
    sys.path.append( package_path )

## getting started

let's try this out on a novel borrowed from the public domain:

In [ ]:
raw_text_path = 'moby_dick.txt'

with open( raw_text_path, 'r') as f:
    raw_text = f.read()

In [ ]:
raw_text[ 0 : 27 ]

the `str` returned from the file read is the starting point for our new datastructure. let's use it to create a new instance:

In [ ]:
from teeth.array import TextStrata

t = TextStrata( raw_text )

initially, a `TextStrata` exposes indices and slices just like the underlying string. the smallest tokens are characters, and slices are just subsequences of these:

In [ ]:
t[ 0 : 27 ]